<a href="https://colab.research.google.com/github/apoorwa46/Spam_Email_Classifier/blob/main/Spam_Email_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/NLP/Preprocessing/mail_data.csv'

df = pd.read_csv(file_path)
df.head()

print(df.head())

texts = df['Message']
labels = df['Category']

# Map labels: 'ham' -> 1, 'spam' -> 0
labels = labels.map({'ham': 1, 'spam': 0})

# Preprocessing function: tokenization and removing stopwords
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(tokens)  # Join the tokens back into a single string

# Apply preprocessing to all texts
texts = texts.apply(preprocess_text)

# Tokenize the text (word embedding preparation)
max_words = 10000  # Maximum number of words to consider in vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences to ensure all inputs have the same length
max_sequence_length = 100  # You can adjust this based on your dataset
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert labels to binary
encoder = LabelEncoder()
y = encoder.fit_transform(labels)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model with an Embedding layer and Simple RNN for BPTT
embedding_dim = 100  # Embedding vector size

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))  # Word embedding layer
model.add(SimpleRNN(64, return_sequences=False))  # Simple RNN with BPTT (Bidirectional is optional)
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

# Predict on test data
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Display the results
# 8. Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive
  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.8625 - loss: 0.3590 - val_accuracy: 0.9785 - val_loss: 0.0827
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - accuracy: 0.9816 - loss: 0.0633 - val_accuracy: 0.9803 - val_loss: 0.0682
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.9952 - loss: 0.0215 - val_accuracy: 0.9803 - val_loss: 0.0782
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.9944 - loss: 0.0212 - val_accuracy: 0.9839 - val_loss: 0.0603
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.9991 - loss: 0.0073 - val_accuracy: 0.9839 - val_loss: 0.0563
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9847 - loss: 0.0524
Test Accuracy: 0.9839
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Accuracy: 0.9838565022421525
Precision: 0.985655737704918
Recall: 0.9958592132505176
F1-score: 0.9907312049433573

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0